## Deliverable 2. Create a Customer Travel Destinations Map.

In [42]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
import sys
sys.path.append("..")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [43]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("..", "Weather_Database", "WeatherPy_database.csv"))
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Qaanaaq,GL,77.48,-69.36,28.54,84,75,4.09,broken clouds
1,1,Kaitangata,NZ,-46.28,169.85,45.00,72,90,5.99,overcast clouds
2,2,Tuktoyaktuk,CA,69.45,-133.04,44.60,87,75,13.87,broken clouds
3,3,Melbu,NO,68.50,14.83,62.60,63,3,9.17,clear sky
4,4,Kapaa,US,22.08,-159.32,78.80,74,20,10.29,few clouds


In [44]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimun temperature you would like for your trip? "))
max_temp = float(input("What is the maximun temperature you would like for your trip? "))

What is the minimun temperature you would like for your trip? 75
What is the maximun temperature you would like for your trip? 90


In [45]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Kapaa,US,22.08,-159.32,78.80,74,20,10.29,few clouds
5,5,Atuona,PF,-9.80,-139.03,78.04,78,8,20.04,light rain
7,7,Ambon,ID,-3.70,128.18,75.20,100,40,4.70,scattered clouds
8,8,Davila,PH,18.47,120.58,80.60,94,75,2.24,broken clouds
9,9,Victoria,HK,22.29,114.16,86.00,74,0,4.00,clear sky


In [46]:
# 4a. Determine if there are any empty rows.
print(len(cities_preferred_df))
cities_preferred_df.count()

299


City_ID                299
City                   299
Country                299
Lat                    299
Lng                    299
Max Temp               299
Humidity               299
Cloudiness             299
Wind Speed             299
Current Description    299
dtype: int64

In [47]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

299


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Kapaa,US,22.08,-159.32,78.80,74,20,10.29,few clouds
5,5,Atuona,PF,-9.80,-139.03,78.04,78,8,20.04,light rain
7,7,Ambon,ID,-3.70,128.18,75.20,100,40,4.70,scattered clouds
8,8,Davila,PH,18.47,120.58,80.60,94,75,2.24,broken clouds
9,9,Victoria,HK,22.29,114.16,86.00,74,0,4.00,clear sky


In [48]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Kapaa,US,78.80,few clouds,22.08,-159.32,
5,Atuona,PF,78.04,light rain,-9.80,-139.03,
7,Ambon,ID,75.20,scattered clouds,-3.70,128.18,
8,Davila,PH,80.60,broken clouds,18.47,120.58,
9,Victoria,HK,86.00,clear sky,22.29,114.16,
11,Mwene-Ditu,CD,78.55,clear sky,-7.00,23.45,
15,Sri Aman,MY,77.00,overcast clouds,1.24,111.46,
18,Zambezi,ZM,79.48,broken clouds,-13.54,23.10,
20,Kalabo,ZM,79.48,broken clouds,-13.54,23.10,
28,Orzhytsya,UA,79.66,overcast clouds,49.79,32.70,


In [49]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = None

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [50]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Kapaa,US,78.80,few clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
5,Atuona,PF,78.04,light rain,-9.80,-139.03,Villa Enata
7,Ambon,ID,75.20,scattered clouds,-3.70,128.18,Swiss-Belhotel Ambon
8,Davila,PH,80.60,broken clouds,18.47,120.58,SITIO 3 DAVILA PASUQUIN
9,Victoria,HK,86.00,clear sky,22.29,114.16,Mini Hotel Central
11,Mwene-Ditu,CD,78.55,clear sky,-7.00,23.45,Hôtel Kamas
15,Sri Aman,MY,77.00,overcast clouds,1.24,111.46,Seri Simanggang Hotel
18,Zambezi,ZM,79.48,broken clouds,-13.54,23.10,Royal Kutachika Lodge
20,Kalabo,ZM,79.48,broken clouds,-13.54,23.10,Royal Kutachika Lodge
29,Santa Cruz,BO,77.00,overcast clouds,-17.80,-63.17,Hotel Cortez


In [51]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [52]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [53]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))